In [17]:
from requests import get, post, delete
import os
import pandas as pd
from sensitive import *


In [18]:
auth = f"Bearer {AUTH_TOKEN}"
headers = {"Accept": "application/json",
           "Content-Type": "application/json", "Authorization": auth}


In [19]:
artists = {}
songs = []


def get_total_song_count():
    playlist_res = get(
        f'https://api.spotify.com/v1/playlists/{PLAYLIST_ID}', headers=headers)
    return playlist_res.json()["tracks"]["total"]


def add_genres(list, genres):
    for genre in genres:
        if genre not in list:
            list.append(genre)

    return list


def get_song_genres(song):
    genres = []
    for artist in song["track"]["artists"]:
        id = artist["id"]
        if id not in artists:
            artist_genres_res = get(
                f'https://api.spotify.com/v1/artists/{id}', headers=headers)
            try:
                artist_genres = artist_genres_res.json()["genres"]
            except Exception:
                print(artist_genres_res.json())
            genres = add_genres(genres, artist_genres)
            artists[artist["name"]] = artist_genres
        else:
            genres = add_genres(genres, artists[id])

    return genres


def song_data(song):
    return song["track"]['id'], song["track"]['name'], song["track"]['popularity'], [artist["name"] for artist in song["track"]["artists"]]


In [20]:
total_song_count = get_total_song_count()

In [21]:


all_genres = []
for offset in range(0, total_song_count, 100):
    all_genres_res = get(
        f'https://api.spotify.com/v1/playlists/{PLAYLIST_ID}/tracks?limit=100&offset={offset}', headers=headers)
    for song in all_genres_res.json()["items"]:
        (id, name, popularity, artist), song_genres = song_data(
            song), get_song_genres(song)
        songs.append({"id": id, "name": name, "artists": artist,
                     "popularity": popularity, "genres": song_genres})
        all_genres = add_genres(all_genres, song_genres)


In [22]:
songs = songs[-total_song_count:]

In [23]:
playlist = pd.DataFrame(data=songs, columns=[
                        'id', 'name', 'artists', 'popularity', 'genres'])


In [24]:
playlist


,id,name,artists,popularity,genres
0,6Z2pe32hFnUd2x5NgO5KFc,Die For You (feat. Dominic Fike),"[Justin Bieber, Dominic Fike]",57,"[canadian pop, pop, alternative pop rock]"
1,4ZLzoOkj0MPWrTLvooIuaa,Get You The Moon (feat. Snøw),"[Kina, Snøw]",83,"[sad lo-fi, sad rap]"
2,4WzhjxvLP95y7AMDy0Atwb,Out Of Love,[Alessia Cara],77,"[canadian contemporary r&b, canadian pop, danc..."
3,4P1L6AniTDJzANaubzWGYs,I'm Not Enough and I'm Sorry,"[Teqkoi, Snøw]",69,"[lo-fi chill, sad lo-fi, sad rap]"
4,6Rf3dWdNQs5LTqFOiavqQr,Canada (feat. Alessia Cara),"[Lauv, Alessia Cara]",61,"[dance pop, electropop, pop, canadian contempo..."
...,...,...,...,...,...
990,77OXCuJjR83b2mnp0HmYMo,False Art,"[Ben Kessler, Lizzy McAlpine]",53,"[gen z singer-songwriter, alt z, boston folk, ..."
991,7dMelypWXvdBG5RpRO1RJU,Purple Candy,[New Friends],54,[]
992,0sEpSjknnHdACZVc7jbmgv,i miss u,[Sammyseven],43,[]
993,01WJWHv12F3nWeXGYsmOOs,Our Way Out,[Nico Collins],57,"[alt z, social media pop]"


In [ ]:
df = pd.DataFrame(data=[{'artists':artists, 'songs':songs, 'all_genres':songs}], columns=['artists', 'songs', 'all_genres'])

df.to_json('util.json')

In [25]:
playlist.to_csv('playlist.csv', header=True, index=None)

In [ ]:
df = pd.read_json('util.json')

artists, songs, all_genres = df['artists'][0], df['songs'][0], df['all_genres'][0]


In [26]:
all_artists = list(artists.keys())
playlist = pd.read_csv('playlist.csv')
playlist

,id,name,artists,popularity,genres
0,6Z2pe32hFnUd2x5NgO5KFc,Die For You (feat. Dominic Fike),"['Justin Bieber', 'Dominic Fike']",57,"['canadian pop', 'pop', 'alternative pop rock']"
1,4ZLzoOkj0MPWrTLvooIuaa,Get You The Moon (feat. Snøw),"['Kina', 'Snøw']",83,"['sad lo-fi', 'sad rap']"
2,4WzhjxvLP95y7AMDy0Atwb,Out Of Love,['Alessia Cara'],77,"['canadian contemporary r&b', 'canadian pop', ..."
3,4P1L6AniTDJzANaubzWGYs,I'm Not Enough and I'm Sorry,"['Teqkoi', 'Snøw']",69,"['lo-fi chill', 'sad lo-fi', 'sad rap']"
4,6Rf3dWdNQs5LTqFOiavqQr,Canada (feat. Alessia Cara),"['Lauv', 'Alessia Cara']",61,"['dance pop', 'electropop', 'pop', 'canadian c..."
...,...,...,...,...,...
990,77OXCuJjR83b2mnp0HmYMo,False Art,"['Ben Kessler', 'Lizzy McAlpine']",53,"['gen z singer-songwriter', 'alt z', 'boston f..."
991,7dMelypWXvdBG5RpRO1RJU,Purple Candy,['New Friends'],54,[]
992,0sEpSjknnHdACZVc7jbmgv,i miss u,['Sammyseven'],43,[]
993,01WJWHv12F3nWeXGYsmOOs,Our Way Out,['Nico Collins'],57,"['alt z', 'social media pop']"


In [27]:
def genres_indexed(genres):
    indexed = []
    for all_genres_x in all_genres:

        continue_outer = False
        for genre in genres:
            index = all_genres_x == genre
            if index:
                continue_outer = True
                indexed.append(int(True))
                break

        if continue_outer:
            continue

        indexed.append(int(False))

    return indexed


def artists_indexed(artists):
    indexed = []
    for all_artists_x in all_artists:

        continue_outer = False
        for genre in artists:
            index = all_artists_x == genre
            if index:
                continue_outer = True
                indexed.append(int(True))
                break

        if continue_outer:
            continue

        indexed.append(int(False))

    return indexed


In [28]:
playlist["genres_indexed"] = [genres_indexed(
    eval(playlist["genres"][x])) for x in range(len(playlist["genres"]))]


In [29]:
playlist.columns = playlist.columns.map(lambda x: x.strip())

In [30]:
playlist["artists_indexed"] = [artists_indexed(
    eval(playlist["artists"][x])) for x in range(len(playlist["artists"]))]


In [31]:
playlist.columns


Index(['id', 'name', 'artists', 'popularity', 'genres', 'genres_indexed',
       'artists_indexed'],
      dtype='object')

## KNN


In [32]:
def knn_prepared_data(df):
    df = df[['id', 'name', 'genres', 'artists',
             'popularity', 'genres_indexed', 'artists_indexed']]
    list = []
    for index in range(len(df['id'])):
        list.append({'id': df['id'][index], 'name': df['name'][index], 'genres': df['genres'][index], 'artists': df['artists'][index],
                    'popularity': df['popularity'][index], 'genres_indexed': df['genres_indexed'][index], 'artists_indexed': df['artists_indexed'][index]})

    return list


In [33]:
song_dict = knn_prepared_data(playlist)

In [34]:
import operator


def getNeighbors(song, K):
    distances = []
    for song_index, song_value in enumerate(song_dict):
        if (song_index != song):
            dist = ComputeDistance(song_dict[song], song_value)
            distances.append((song_index, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors


In [35]:
def list_distance(a, b):
    distance = 0
    for item_a, item_b in list(zip(a, b)):
        distance += 0 if item_a == item_b else 0.2 if (len(a) < 400) else 0.1

    return distance

In [36]:
def ComputeDistance(a, b):
    genres_distance = list_distance(a['genres_indexed'], b['genres_indexed'])
    artists_distance = list_distance(a['artists_indexed'], b['artists_indexed'])
    popularity_distance = abs(a['popularity'] - b['popularity'])
    
    return genres_distance + artists_distance * 0.4 + popularity_distance * 0.003


## Song options

### Choose one of these to get the 50 closest songs to it


In [37]:
# Set the song name below
top_5 = get('https://api.spotify.com/v1/me/top/tracks?time_range=short_term&limit=5', headers=headers).json()
song_name = "Summer Nights"
# top_5_names = map(lambda song: song['name'],top_5['items'])
# for i in top_5_names:
#     if i in playlist['name']:
#         song_name = i
#         break

In [38]:
K = 50
item = playlist[[playlist['name'][x] ==
                 song_name for x in range(len(playlist['name']))]]
index = item.index[0]
neighbors = getNeighbors(index, K)
print(song_dict[index]['name'], song_dict[index]['artists'],
      song_dict[index]['genres'], song_dict[index]['popularity'])
for neighbor, index in zip(neighbors, range(1, K + 1)):
    print(
        f'{index}. {song_dict[neighbor]["name"]} \t {song_dict[neighbor]["artists"]} \t {song_dict[neighbor]["genres"]} \t {song_dict[neighbor]["popularity"]}')


Summer Nights ['Ebmusic', 'bobbyxyz', 'Kina'] ['sad lo-fi', 'sad rap'] 15
1. What's The Point? 	 ['Snøw'] 	 ['sad lo-fi', 'sad rap'] 	 34
2. i dont wanna 	 ['"224thotty"', 'Snøw'] 	 ['sad lo-fi', 'sad rap'] 	 7
3. Baby You're Worth It 	 ['Kina'] 	 ['sad lo-fi', 'sad rap'] 	 70
4. as long as i have you 	 ['Foster'] 	 ['sad lo-fi', 'sad rap'] 	 50
5. If You Leave 	 ['Sky McCreery'] 	 ['sad lo-fi', 'sad rap'] 	 62
6. Get You The Moon (feat. Snøw) 	 ['Kina', 'Snøw'] 	 ['sad lo-fi', 'sad rap'] 	 83
7. Don't Save Me 	 ['ill Nicky'] 	 ['sad rap'] 	 23
8. the reason i'm breathing 	 ['sadeyes'] 	 ['emo rap', 'sad lo-fi', 'sad rap'] 	 27
9. moi 	 ['sadeyes'] 	 ['emo rap', 'sad lo-fi', 'sad rap'] 	 27
10. headaches 	 ['Zac Greer'] 	 ['sad rap'] 	 1
11. I Am so Sad... I Am so Very Very Sad 	 ['Alrahim Wright III'] 	 ['lo-fi chill', 'sad lo-fi', 'sad rap'] 	 0
12. Just a Little Bit Lonely 	 ['yaeow'] 	 ['lo-fi chill', 'sad lo-fi', 'sad rap'] 	 0
13. crystal clear w/ guardin 	 ['sadeyes'] 	 ['emo ra

In [39]:
def playlist_exists(name):
    request = get('https://api.spotify.com/v1/me/playlists', headers=headers).json()

    playlists = list(map(lambda playlist: (playlist['id'], playlist['name']), request['items']))

    for playlist in playlists:
        if playlist[1] == name:
            return playlist[0]
        
    return False

In [40]:
import json
playlist_name = f"{song_name!r} Related"
new_id = ""
playlist_id_found = playlist_exists(playlist_name)
if playlist_id_found:
    new_id = playlist_id_found
    
    playlist_tracks = list(map(lambda track: {'uri': track['track']['uri']}, get(f'https://api.spotify.com/v1/playlists/{new_id}/tracks', headers=headers).json()['items']))

    delete_json = delete(f'https://api.spotify.com/v1/playlists/{new_id}/tracks',  headers=headers, data=json.dumps({"tracks": playlist_tracks})).json()
    print(delete_json)
else:
    data = {"name": playlist_name, "description": f"Songs related to {song_name!r}", "public": False}
    playlist_creation = post(f'https://api.spotify.com/v1/users/{USER_ID}/playlists', headers=headers, data=json.dumps(data))
    new_id = playlist_creation.json()['id']

In [41]:
item = playlist[[playlist['name'][x] == song_name for x in range(len(playlist['name']))]]
index = item.index[0]
song_uris = f'spotify:track:{song_dict[index]["id"]}'
for neighbor in neighbors:
    song_uris += f',spotify:track:{song_dict[neighbor]["id"]}'

add_songs_req = post(f'https://api.spotify.com/v1/playlists/{new_id}/tracks?uris={song_uris}', headers=headers, data=json.dumps({}))
add_songs_req.json()

{'snapshot_id': 'MywzZjMwMzAxZTVhMjQzNGM1MmZjNDg2MjI1OTAyNzJiOWIwMmFhMGQ2'}